# Things to remember about AWS DynamoDB
> This short compendium will help in preparing for the AWS certification exams

- toc: true
- badges: true
- categories: [aws, certification, dynamodb]

# Indexes

A secondary index allows you to perform more flexible querying on DynamoDB. It allows you to query on an attribute which is not the primary key. This can be done using 

- Global Secondary Index
- Local Secondary Index

You select the columns you want to include in the index and run your searches on the index instead of the entire dataset.


## Local Secondary Index

    Limitation: 
        Can only be created when you're creating the table.
Has the same partition key as your original table, but a different sort key.

## Global Secondary index

    Advantage:
        Can be created even after creation of the table.
        It allows you to use a different partition key as well as the sort key than your main table
        Gives a totally different view of the table than the original table created (because of the above point)